<a href="https://colab.research.google.com/github/mjkimcs/portfolio/blob/main/%EB%94%A5%EB%9F%AC%EB%8B%9D/fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
###
! [ -e /content ] && pip install -Uqq fastai  # upgrade fastai on colab

     |████████████████████████████████| 188 kB 4.4 MB/s 
     |████████████████████████████████| 56 kB 1.5 MB/s 


In [ ]:
###
from fastai.tabular.all import *

In [ ]:
path = untar_data(URLs.ADULT_SAMPLE)
path.ls()

(#3) [Path('/root/.fastai/data/adult_sample/export.pkl'),Path('/root/.fastai/data/adult_sample/models'),Path('/root/.fastai/data/adult_sample/adult.csv')]

In [ ]:
###
df = pd.read_csv(path/'adult.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,>=50k
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,>=50k
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,<50k
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,>=50k
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,<50k


In [ ]:
df.shape

(32561, 15)

In [ ]:
###
# 이후에 dls(train셋)로 할당될 예정

splits = RandomSplitter(valid_pct=0.2)(range_of(df))

to = TabularPandas(df, procs=[Categorify, FillMissing, Normalize],
                   cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race'],
                   cont_names = ['age', 'fnlwgt', 'education-num'],
                   y_names='salary',
                   splits=splits)

In [ ]:
to.xs.shape

(32561, 10)

In [ ]:
to.xs.head()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
13434,5,12,3,4,1,5,1,0.839207,-1.120717,-0.422546
30195,5,12,5,2,2,5,1,-1.066129,0.193774,-0.422546
26668,1,11,3,1,1,5,1,2.158286,1.534448,2.315975
8167,5,12,5,4,2,5,1,1.059054,0.246340,-0.422546
531,5,10,5,0,2,5,2,-0.699718,1.931436,-0.031329


In [ ]:
to.xs.iloc[[0,1]]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
13434,5,12,3,4,1,5,1,0.839207,-1.120717,-0.422546
30195,5,12,5,2,2,5,1,-1.066129,0.193774,-0.422546


In [ ]:
to.ys.head(2)

,salary
13434,1
30195,0


Now we can build our `DataLoaders` again:

In [ ]:
###
dls = to.dataloaders(bs=64)

In [ ]:
dls.xs.shape

(26049, 10)

In [ ]:
26049/32561

0.8000061423174963

In [ ]:
dls.xs.head()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
13434,5,12,3,4,1,5,1,0.839207,-1.120717,-0.422546
30195,5,12,5,2,2,5,1,-1.066129,0.193774,-0.422546
26668,1,11,3,1,1,5,1,2.158286,1.534448,2.315975
8167,5,12,5,4,2,5,1,1.059054,0.246340,-0.422546
531,5,10,5,0,2,5,2,-0.699718,1.931436,-0.031329


In [ ]:
dls.show_batch()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Self-emp-not-inc,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,False,41.000000,287036.998918,9.0,<50k
1,Private,HS-grad,Married-civ-spouse,Sales,Husband,Asian-Pac-Islander,False,62.000000,77883.994303,9.0,<50k
2,Private,Masters,Married-civ-spouse,Exec-managerial,Husband,White,False,40.000000,160368.999314,14.0,>=50k
3,Local-gov,Masters,Never-married,Prof-specialty,Own-child,White,False,25.000000,276249.000993,14.0,<50k
4,Private,Assoc-acdm,Never-married,Sales,Not-in-family,White,False,39.000000,98947.999027,12.0,<50k
5,Private,HS-grad,Never-married,Adm-clerical,Other-relative,White,False,36.000000,182074.000160,9.0,<50k
6,Private,HS-grad,Married-civ-spouse,Other-service,Husband,White,False,61.000001,357436.995126,9.0,<50k
7,Private,HS-grad,Separated,Craft-repair,Not-in-family,Black,False,41.000000,299504.996996,9.0,<50k
8,?,12th,Married-civ-spouse,?,Other-relative,White,False,18.999999,200790.000054,8.0,>=50k
9,Self-emp-inc,HS-grad,Never-married,Craft-repair,Not-in-family,White,False,32.000000,281029.998388,9.0,<50k


In [ ]:
###
# 학습
learn = tabular_learner(dls, metrics=accuracy)
learn.fit_one_cycle(1)

epoch,train_loss,valid_loss,accuracy,time
0,0.362736,0.365691,0.830467,00:07


In [ ]:
row, clas, probs = learn.predict(df.iloc[0])

In [ ]:
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,Private,Assoc-acdm,Married-civ-spouse,#na#,Wife,White,False,49.0,101320.0002,12.0,>=50k


In [ ]:
clas, probs

(tensor(1), tensor([0.3092, 0.6908]))

In [ ]:
probs[0].item()

0.30916380882263184

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             32561 non-null  int64  
 1   workclass       32561 non-null  object 
 2   fnlwgt          32561 non-null  int64  
 3   education       32561 non-null  object 
 4   education-num   32074 non-null  float64
 5   marital-status  32561 non-null  object 
 6   occupation      32049 non-null  object 
 7   relationship    32561 non-null  object 
 8   race            32561 non-null  object 
 9   sex             32561 non-null  object 
 10  capital-gain    32561 non-null  int64  
 11  capital-loss    32561 non-null  int64  
 12  hours-per-week  32561 non-null  int64  
 13  native-country  32561 non-null  object 
 14  salary          32561 non-null  object 
dtypes: float64(1), int64(5), object(9)
memory usage: 3.7+ MB


To get prediction on a new dataframe, you can use the `test_dl` method of the `DataLoaders`. That dataframe does not need to have the dependent variable in its column.

In [ ]:
###
## 학습할 때 종속변수는 "<50k" 이러한 type이어야 하고, 이 시점에서 숫자로 매핑
mapping = {"<50k":0, ">=50k": 1}
for i in [df]:
    i['salary'] = i['salary'].map(mapping)
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,49,Private,101320,Assoc-acdm,12.0,Married-civ-spouse,NaN,Wife,White,Female,0,1902,40,United-States,1
1,44,Private,236746,Masters,14.0,Divorced,Exec-managerial,Not-in-family,White,Male,10520,0,45,United-States,1
2,38,Private,96185,HS-grad,NaN,Divorced,NaN,Unmarried,Black,Female,0,0,32,United-States,0
3,38,Self-emp-inc,112847,Prof-school,15.0,Married-civ-spouse,Prof-specialty,Husband,Asian-Pac-Islander,Male,0,0,40,United-States,1
4,42,Self-emp-not-inc,82297,7th-8th,NaN,Married-civ-spouse,Other-service,Wife,Black,Female,0,0,50,United-States,0


In [ ]:
###
dls.xs.shape

(26049, 10)

In [ ]:
###
X_tst = df[26049:].copy()
y_tst = X_tst["salary"]
X_tst.drop(["salary"], axis=1, inplace=True)

In [ ]:
###
dl = learn.dls.test_dl(X_tst)
preds = learn.get_preds(dl=dl)

In [ ]:
dl.xs.head()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
26049,8,13,3,5,1,5,1,1.425464,0.856169,1.533540
26050,5,16,1,5,5,5,1,-0.626436,-1.494660,-0.031329
26051,5,10,7,9,2,5,1,2.597979,-0.086683,1.142323
26052,5,16,6,9,2,5,1,-0.406590,-0.206968,-0.031329
26053,5,12,3,9,1,5,1,-0.553154,0.318792,-0.422546


In [ ]:
preds[0]

tensor([[0.1305, 0.8695],
        [0.9524, 0.0476],
        [0.9357, 0.0643],
        ...,
        [0.4704, 0.5296],
        [0.7820, 0.2180],
        [0.7620, 0.2380]])

In [ ]:
preds[0][0][0].item()

0.13053463399410248

In [ ]:
###
preds[0][:,1].numpy()

array([0.86946535, 0.04762873, 0.06432542, ..., 0.5295566 , 0.21802127,
       0.23799141], dtype=float32)

In [ ]:
###
preds = (preds[0].numpy()[:, 1] >= 0.5).astype('int')
preds

array([1, 0, 0, ..., 1, 0, 0])

In [ ]:
###
from sklearn.metrics import classification_report

print(classification_report(y_pred=preds, y_true=y_tst))

              precision    recall  f1-score   support

           0       0.88      0.92      0.90      4982
           1       0.69      0.58      0.63      1530

    accuracy                           0.84      6512
   macro avg       0.78      0.75      0.77      6512
weighted avg       0.83      0.84      0.84      6512



In [ ]:
row, clas, probs = learn.predict(X_tst.iloc[0])
row.show()

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num,salary
0,State-gov,Masters,Married-civ-spouse,Exec-managerial,Husband,White,False,58.000001,279877.997925,14.0,>=50k


In [ ]:
###
learn.export(fname="./mo.pkl", pickle_module=pickle, pickle_protocol=2)
learn = load_learner("./mo.pkl", cpu=True, pickle_module=pickle)

In [ ]:
###
X_trn = df[:26049].copy()
y_trn = X_trn["salary"]
X_trn.drop(["salary"], axis=1, inplace=True)

In [ ]:
###
dl = learn.dls.test_dl(X_trn)
preds = learn.get_preds(dl=dl)
preds

(tensor([[0.3092, 0.6908],
         [0.4755, 0.5245],
         [0.9552, 0.0448],
         ...,
         [0.8872, 0.1128],
         [0.8537, 0.1463],
         [0.8896, 0.1104]]), None)

In [ ]:
###
p_val = preds[0].numpy()

In [ ]:
###
dl = learn.dls.test_dl(X_tst)
preds2 = learn.get_preds(dl=dl)
preds2

(tensor([[0.1305, 0.8695],
         [0.9524, 0.0476],
         [0.9357, 0.0643],
         ...,
         [0.4704, 0.5296],
         [0.7820, 0.2180],
         [0.7620, 0.2380]]), None)

In [ ]:
preds2[0].numpy()

array([[0.13053463, 0.86946535],
       [0.95237124, 0.04762873],
       [0.9356746 , 0.06432542],
       ...,
       [0.47044346, 0.5295566 ],
       [0.7819787 , 0.21802127],
       [0.76200855, 0.23799141]], dtype=float32)

In [ ]:
###
p_tst = preds2[0].numpy()

In [ ]:
###
data_dir = Path('./')
algo_name = 'fastai'
feature_name = 'stacking1'
model_name = f'{algo_name}_{feature_name}'

p_val_file = data_dir / f'{model_name}.val.csv'
p_tst_file = data_dir / f'{model_name}.tst.csv'

np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [ ]:
###
a = np.loadtxt(data_dir / f'{model_name}.tst.csv', delimiter=',')
a

array([[0.130535, 0.869465],
       [0.952371, 0.047629],
       [0.935675, 0.064325],
       ...,
       [0.470443, 0.529557],
       [0.781979, 0.218021],
       [0.762009, 0.237991]])

In [ ]:
### (참고)a와 똑같은 b를 만들었었음
X_trn1 = []
for i in [a, b]:
  X_trn1.append(i)
X_trn1 = np.hstack(X_trn1)
X_trn1

array([[0.130535, 0.869465, 0.130535, 0.869465],
       [0.952371, 0.047629, 0.952371, 0.047629],
       [0.935675, 0.064325, 0.935675, 0.064325],
       ...,
       [0.470443, 0.529557, 0.470443, 0.529557],
       [0.781979, 0.218021, 0.781979, 0.218021],
       [0.762009, 0.237991, 0.762009, 0.237991]])

## `fastai` with Other Libraries

As mentioned earlier, `TabularPandas` is a powerful and easy preprocessing tool for tabular data. Integration with libraries such as Random Forests and XGBoost requires only one extra step, that the `.dataloaders` call did for us. Let's look at our `to` again. It's values are stored in a `DataFrame` like object, where we can extract the `cats`, `conts,` `xs` and `ys` if we want to:

In [ ]:
to.xs[:3]

,workclass,education,marital-status,occupation,relationship,race,education-num_na,age,fnlwgt,education-num
25387,5,16,3,5,1,5,1,0.471582,-1.467756,-0.030907
16872,1,16,5,1,4,5,1,-1.215622,-0.649792,-0.030907
25852,5,16,3,5,1,5,1,1.865358,-0.218915,-0.030907


Now that everything is encoded, you can then send this off to XGBoost or Random Forests by extracting the train and validation sets and their values:

In [ ]:
X_train, y_train = to.train.xs, to.train.ys.values.ravel()
X_test, y_test = to.valid.xs, to.valid.ys.values.ravel()

In [ ]:
to.xs.shape

(32561, 10)

In [ ]:
X_train.shape

(26049, 10)

In [ ]:
X_test.shape

(6512, 10)